$\def \dot #1#2{\left\langle #1, #2 \right\rangle}$
$\def \adot #1#2{\left\langle #1, #2 \right\rangle}$
$\def \cD {\mathcal{D}}$
$\def \cM {\mathcal{M}}$
$\def \bc {\mathbf{c}}$
$\def \bv {\mathbf{v}}$
$\def \bG {\mathbf{G}}$

### Multispace decomposition of the two dimensional field solution

$y\in\mathbb{R}^2$, $a(y) = y_1 \chi_{D_1}(x) + y_2 \chi_{D_2}(x)$, and $D_1 = [0,1/2) \times [0,1]$ and $D_2 = [1/2, 1] \times [0,1]$, and $\chi_{D_1}$, $\chi_{D_2}$ are the indicator functions on $D_1$, $D_2$.

We have 
$$
a(y) = \bar{a} + c(y_1 \chi_{D_1} + y_2 \chi_{D_2})
$$

We define the operators $A_0$, $A_1$ and $A_2$ in the inner product
$$
\dot{A_0 u}{v}_V = \int_D \nabla u \cdot \nabla v \, \mathrm{d} x
$$
and
$$
\dot{A_1 u}{v}_V = \int_{D} \chi_{D_1} \nabla u \cdot \nabla v \, \mathrm{d} x = \int_{D_1} \nabla u \cdot \nabla v \, \mathrm{d} x
$$
and similar for $A_2$. Recall that $V=H_0^1(D)$. Now let us assume that both $a$ is constant. We can re-write the PDE problem in its weak form as

$$
\left ( \bar{a}A_0 + c y_1 A_1 + c y_2 A_2 \right) u = f
$$

The essence of the is to see that we can pull out the "scale" of the field out the front, and we are left with one more parameter which is the amount of difference in the field between $D_1$ and $D_2$, that is,
$$
a(y_1, y_2) = a(y_m, y_d) = y_m \left( 1 + y_d (\chi_{D_1} - \chi_{D_2}) \right)
$$
where we have defined 
$$
y_m = \bar{a} + c\frac{y_1 + y_2}{2} \quad \text{and} \quad y_d = c \frac{y_1-y_2}{2} y_m^{-1}
$$

Now, we write $u_0\in H_0^1(D)$, $u_1\in H_0^1(D_1)$ and $u_2 \in H_0^1(D_2)$ for the solutions of 
$$
A_0 u_0 = f \quad A_1 u_1 = \chi_{D_1} f \quad A_2 u_2 = \chi_{D_2} f
$$

__Then the solution is given by__

$$
\large
u = \frac{1}{y_m} \left( u_0 - y_d \left( \frac{u_1}{1+y_d} - \frac{u_2}{1-y_d}\right)\right)
$$

I show this in a write-up somehwere. Point is, it's a 3-dimensional manifold (the solution is composed of only $u_0, u_1$ and $u_2$ indexed by two parameters, $y_1$ and $y_2$.

In [1]:
import numpy as np
import scipy as sp
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import pdb

import sys
sys.path.append("../../")
import pyApproxTools as pat
importlib.reload(pat)

%matplotlib inline

def make_soln(points, fem_div, a_bar=1.0, c=0.5, f=1.0, verbose=False):
    
    solns = []
    fields = []

    for p in points:
        field = pat.PWConstantSqDyadicL2(a_bar + c * p.reshape((2,2)))
        fields.append(field)
        # Then the fem solver (there a faster way to do this all at once? This will be huge...
        fem_solver = pat.DyadicFEMSolver(div=fem_div, rand_field = field, f = 1)
        fem_solver.solve()
        solns.append(fem_solver.u)
        
    return solns, fields

def make_2d_soln(points, fem_div, a_bar=1.0, c=0.5, f=1.0, verbose=False):
    
    solns = []
    fields = []

    for p in points:
        field = pat.PWConstantSqDyadicL2(a_bar + c * np.repeat(p[:,np.newaxis], 2, axis=1).T)
        fields.append(field)
        # Then the fem solver (there a faster way to do this all at once? This will be huge...
        fem_solver = pat.DyadicFEMSolver(div=fem_div, rand_field = field, f = 1)
        fem_solver.solve()
        solns.append(fem_solver.u)
        
    return solns, fields

fem_div = 7
a_bar = 0.1
c = 2.0

def diffusion_pde(points):
    solns, fields = make_soln(points, fem_div=fem_div, a_bar=a_bar, c=c)
    return solns
    
def diffusion_2d_pde(points):
    solns, fields = make_2d_soln(points, fem_div=fem_div, a_bar=a_bar, c=c)
    return solns

def soln_loc_2d_pde(points):
    ym = a_bar + points.mean(axis=1)
    yd = c * (points[:,0] - points[:,1]) / (2*ym)
    
    soln_loc = np.zeros((points.shape[0], 3))
    soln_loc[:,0] = 1.0 / ym
    soln_loc[:,1] = - yd / (ym * (1 + yd))
    soln_loc[:,2] = yd / (ym * (1 - yd))
    
    return soln_loc

# For the sake of testing we make these solutions
y1 = np.array([[1, 1e10]])
y2 = np.array([[1e10, 1]])
u0, a0 = make_2d_soln(np.array([[1,1]]), fem_div, a_bar=0, c=1)
u1, a1 = make_2d_soln(y1, fem_div, a_bar=0, c=1)
u2, a2 = make_2d_soln(y2, fem_div, a_bar=0, c=1)

## ...So, what are we doing?

Well, lets just say we have any old parametric function $u(y)$ that defines a set of points $\cM = \{u(y) : y\in Y \}$. Doesn't have to be the PDE. Lets assume $Y = [0,1]^d$ and we are interested in the push forward measure of the uniform measure on $Y$, which we call $\mu$ and is supported on $\cM$.

Our first step is to construct a PCA approximation $\mu_1$ and calculate the lifting operator.

Then we look at the $d$ different splittings available. There'sa variety of choice criteria available.

Then we create the next splitting, and calculate the $d$ possible splits in that cell...

At each step we have $(k+1)d$ possible criteria to search from.

We should also have the multilinear estimator built.

Oh my god this is doing my head in.

In [140]:
# Now we need some tree structure to keep track of the splitting tree    
from anytree import NodeMixin, RenderTree
import itertools

def PCA(U, eig_tol=1e-12):
    
    u_mean = U @ (np.ones(U.n) / U.n)
    u_mean_cor = np.zeros(U.n)
    for i, u in enumerate(U):
        u_mean_cor[i] = u_mean.dot(u)
    u_mnsq = u_mean.dot(u_mean)
    
    mean_free_G = np.copy(U.G)
    for i in range(U.n):
        for j in range(U.n):
            mean_free_G[i,j] -= (u_mean_cor[i] + u_mean_cor[j] + u_mnsq)
    
    sig, V = sp.linalg.eigh(mean_free_G)

    V = V[:,::-1]
    sig = sig[::-1]
    V = V[:,sig > eig_tol]
    sig = sig[sig > eig_tol]

    PCA_Basis = U @ V @ np.diag(np.sqrt(1.0/sig))

    return PCA_Basis, sig, u_mean

class ParamPWBasis(pat.PWBasis):
    """ Just a class that has the parameters associated with the basis at the same time """
    def __init__(self, vecs=None, ys=None, G=None, values_flat=None, pre_allocate=0, file_name=None):
        super().__init__(vecs=vecs, G=G, values_flat=values_flat, pre_allocate=pre_allocate, file_name=file_name)
        
        if ys is not None:
            assert ys.shape[0] == len(vecs)
            self.ys = ys
        
    def add_vec(self, vec, y):
        """ Add just one vector, so as to make the new Grammian calculation quick """
        super().add_vec(vec)
        if ys is not None:
            self.ys = np.vstack((self.ys, y))

    def append(self, other, ys):
        """ Add just one vector, so as to make the new Grammian calculation quick """
        super().append(other)
        if ys is not None:
            self.ys = np.vstack((self.ys, ys))
    
    def mask(self, y_range):
        return np.all((self.ys > y_range[:,0]) & (self.ys <= y_range[:,1]), axis=-1)
 
    def __getitem__(self, idx):
        if self._is_herm_trans:
            raise IndexError(self.__name__ + ': can not access rows of Hermitian of basis')

        if isinstance(idx, int):
            return self._vecs[idx]

        elif isinstance(idx, slice):
            sub = type(self)(vecs=self._vecs[idx], ys=self.ys[idx], values_flat=self._values_flat[:,:,idx])
            if not np.any(self._vec_is_new):
                sub._G = self._G[idx, idx]
            return sub

        elif hasattr(idx, '__len__') and len(idx) == len(self): 
            # NOT CONFIDENT THAT THIS WORKS... but in this case idx is a boolean mask...
            sub = type(self)(vecs=list(itertools.compress(self._vecs, idx)), ys=self.ys[idx], values_flat=self._values_flat[:,:,idx])
            if not np.any(self._vec_is_new):
                sub._G = self._G[idx,idx]
            return sub

        else:
            raise TypeError(self.__class__.__name__ + ': idx type incorrect')


class LocalNormalEstimator(NodeMixin):
    
    def __init__(self, U, f, y_range, min_N=500, parent=None):
        
        self.U = U

        self.y_range = y_range
        self.d = y_range.shape[1]
        self.f = f
        self._min_N = min_N

        self._mask = U.mask(self.y_range)

        if self._mask.sum() < self._min_N:
            ys_new = np.random.random((self._min_N - self._mask.sum(), self.d)) * (y_range[:, 1] - y_range[:, 0]) + y_range[:, 0]
            U_new = self.f(ys_new)
            print(U_new, ys_new)
            self.U.append(U_new, ys_new)
            self._mask = U.mask(self.y_range)

        self._local_U = self.U[self._mask]
        self.Phi, self.sigma_sq = PCA(self._local_U)
        
        self.parent = parent
        
    def cond_variance(self, Wm):
        
        self.GW = Wm.T @ self.Phi @ np.diag(np.sqrt(self.sigma_sq))
        self.r = np.linalg.matrix_rank(self.GW)
        
        Psi_til, S_til, Phi_til = np.linalg.svd(self.GW)
        Phi_perp = Phi_til[r:].T
        
        self._cond_variance = np.trace(Phi_perp.T @ np.diag(self.sigma_sq) @ Phi_perp)
        
        return self._cond_variance
        
    def variance(self):
        return np.sqrt(self.sigma_sq.prod())
        
    def children_angle(self, angle_tol=1e-10):
        # Some weird criteria I'm imagining to choose the children that have the biggest difference
        # in span of the PCA space...
        
        self.child_angles = np.zeros((self.d, 3))
        self.ranks = np.zeros((self.d, 3))
        for i, (child_l, child_r) in enumerate(zip(self.cand_child_l, self.cand_child_r)):

            print(self.sigma_sq)
            print(child_l.sigma_sq)
            print(child_r.sigma_sq)
            print(self.sigma_sq[0] / self.sigma_sq[-1], child_l.sigma_sq[0] / child_l.sigma_sq[-1], child_r.sigma_sq[0] / child_r.sigma_sq[-1])
            print(self.sigma_sq.prod(), child_l.sigma_sq.prod(), child_r.sigma_sq.prod())
            print(self.sigma_sq.prod()**(1.0/len(self.sigma_sq)), child_l.sigma_sq.prod()**(1.0/len(child_l.sigma_sq)), child_r.sigma_sq.prod()**(1.0/len(child_r.sigma_sq)))
            G_p_l = (self.Phi @ np.diag(np.sqrt(self.sigma_sq))).H @ child_l.Phi @ np.diag(np.sqrt(child_l.sigma_sq))
            S_p_l = sp.linalg.svd(G_p_l, compute_uv=False)

            print(G_p_l)
            print(S_p_l)
            
            G_p_r = (self.Phi @ np.diag(np.sqrt(self.sigma_sq))).H @ child_r.Phi @ np.diag(np.sqrt(child_r.sigma_sq))
            S_p_r = sp.linalg.svd(G_p_r, compute_uv=False)
            print(G_p_r)
            print(S_p_r)
            
            G_l_r = (child_l.Phi @ np.diag(np.sqrt(child_l.sigma_sq))).H @ child_r.Phi @ np.diag(np.sqrt(child_r.sigma_sq))
            S_l_r = sp.linalg.svd(G_l_r, compute_uv=False)
            print(G_l_r)
            print(S_l_r)
            
            self.child_angles[i, 0] = S_p_l[-1]
            self.child_angles[i, 1] = S_p_r[-1]
            self.child_angles[i, 2] = S_l_r[-1]
        
            self.ranks[i, 0] = np.linalg.matrix_rank(G_p_l)
            self.ranks[i, 1] = np.linalg.matrix_rank(G_p_r)
            self.ranks[i, 2] = np.linalg.matrix_rank(G_l_r)
            
    def children_W_2(self):
        self.W_adj = np.zeros(self.d)
        
        for i, (child_l, child_r) in enumerate(zip(self.cand_child_l, self.cand_child_r)):
            
            lr_CG = child_l.Phi.H @ child_r.Phi
            
            W_adj[i] = np.trace(np.diag(np.sqrt(child_l.sigma_sq)) @ lr_CG @ np.diag(child_r.sigma_sq) \
                     @ lr.CG.T @ np.diag(np.sqrt(child_l.sigma_sq))
            
        
    
    def generate_potential_children(self):
        # Loop through all d dimensions, do the splitting (by creating) and save
        
        self.cand_child_l = []
        self.cand_child_r = []ts
        for i in range(d):
            
            y_range_l = np.copy(self.y_range)
            y_range_l[i,1] = self.y_range[i,:].mean()
            y_range_r = np.copy(self.y_range)
            y_range_r[i,0] = self.y_range[i,:].mean()
            
            # The children are set up with no parent at first. They are "detached" for now. The ones we choose will become attached
            self.cand_child_l.append(LocalNormalEstimator(self.U, self.f, y_range_l, min_N=self._min_N, parent=None))
            self.cand_child_r.append(LocalNormalEstimator(self.U, self.f, y_range_r, min_N=self._min_N, parent=None))
        
    def split(self):
        
        self.children_angle()

In [141]:
N = 100
d = 2

y_range = np.zeros((d,2))
y_range[:,1] = 1

np.random.seed(1)
points = np.random.random((N, d)) * (y_range[:, 1] - y_range[:, 0]) + y_range[:, 0]
us = diffusion_2d_pde(points)
U = ParamPWBasis(vecs=us, ys=points)

In [142]:
y_range = np.zeros((d,2))
y_range[:,1] = 1

l = LocalNormalEstimator(U, diffusion_2d_pde, y_range, min_N=50)

In [143]:
print(l.variance())
l.generate_potential_children()
l.children_angle()

0.048646783723948324
[<pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107ea3c50>, <pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107e772e8>, <pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x102c28668>] [[0.97508806 0.55665319]
 [0.95780317 0.64156621]
 [0.69500386 0.48599067]]
[<pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107eac748>, <pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107eac390>, <pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107eac1d0>, <pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107eacdd8>, <pyApproxTools.pw_vector.PWLinearSqDyadicH1 object at 0x107eaccc0>] [[0.60431048 0.27477396]
 [0.92618143 0.45936672]
 [0.39487561 0.48163126]
 [0.17395567 0.06316476]
 [0.13507916 0.25283108]]


IndexError: index 106 is out of bounds for axis 1 with size 106

In [136]:
len(U.ys)

103

In [ ]:
points_new = np.random.random((10, d)) * (y_range[:, 1] - y_range[:, 0]) + y_range[:, 0]
np.vstack((points, points_new)).shape

In [ ]:
y[:,:] < r[:,0]
print(y)
r[:,0] < y

In [ ]:
a=[3,4]
b = [4,4]
a.append(b[:])

In [11]:
U.n
points.shape

(100, 2)